[Moved to a new repo](https://github.com/aws-samples/ml-specialized-hardware/tree/main/tutorials/04_ImageGenerationWithSDXL)

### Delete the endpoint (manually or scheduled)

In [ ]:
import time

last_cell_timestamp = time.time()

If you're running the notebook with "Run All Cells" command, the above cell and the below cell will be executed automatically one by one and the endpoint will stay active for you to experiment with. It will incur additional hourly charges.

However, if you run the below cell manually again after some time depending on the `grace_period_seconds` variable, the endpoint will be gracefully deleted and no further charges will occur.

If you forget to delete the endpoint manually, but will keep the notebook kernel instance up and running, the endpoint will be deleted by the background thread depending on the `auto_delete_hours` variable.

In [ ]:
import threading
import botocore.exceptions

seconds_since_last_cell_timestamp = int(time.time() - last_cell_timestamp)
grace_period_seconds = 30
auto_delete_hours = 4.0

def delete_endpoint():
    try:
        predictor.delete_endpoint(delete_endpoint_config=False)
    except botocore.exceptions.ClientError as e:
        # Most likely, already deleted
        print(e)
        pass
    print("Endpoint has been deleted.")
    
def schedule_auto_delete():
    global auto_delete_timer
    try:
        print("Cancelling previous timer...")
        auto_delete_timer.cancel()
    except NameError:
        print("Timer hasn't been yet defined.")
        print(f"Scheduling the automatic deletion in {auto_delete_hours} hours.")
    else:
        print("Timer is cancelled.")
        print(f"Re-scheduling the automatic deletion in {auto_delete_hours} hours.")
    auto_delete_timer = threading.Timer(auto_delete_hours * 60 * 60, delete_endpoint)
    auto_delete_timer.start()    

print("Checking notebook for automated run...")
print(f"Seconds since last cell exectution: {seconds_since_last_cell_timestamp}")
if seconds_since_last_cell_timestamp > grace_period_seconds:
    print("Cell is executed manually. Deleting endpoint.")
    delete_endpoint()
else:
    print("Cell is executed automatically. Skipping endpoint delete. Don't forget to run the cell again to delete endpoint manually.")
    schedule_auto_delete()

If you don't want to delete the endpoint right now, but want to extend the automatic deletion instead, run the below cell manually. The countdown for automatic deletion will start over.

In [ ]:
seconds_since_last_cell_timestamp = int(time.time() - last_cell_timestamp)
print(f"Seconds since last cell exectution: {seconds_since_last_cell_timestamp}")

if seconds_since_last_cell_timestamp > grace_period_seconds:
    print(f"Cell is executed manually. Re-scheduling the timer.")
    schedule_auto_delete()
else:
    print("Cell is executed automatically. Skipping re-scheduling. Run the cell again to re-schedule deletion.")